In [1]:
import functools

import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
# Download Data using tf.keras.utils.get_file

TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [3]:
# Make numpy values easier to read.

np.set_printoptions(precision = 3, suppress = True)

In [4]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [5]:
# Read CSV Files onto a dataset
# tf.data.experimental.make_csv_dataset returns tensors as a tuple (features, labels)

def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size = 5, 
      label_name=LABEL_COLUMN,
      na_value = "?",
      num_epochs = 1,
      ignore_errors = True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [6]:
# tf.data.dataset class, take(count) function returns a dataset with count number of elements

def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [7]:
# Mixed Dataset (dtypes) 

show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [16. 28. 28. 45. 62.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [0 0 0 0 0]
fare                : [26.     8.05   7.775  6.975 10.5  ]
class               : [b'Second' b'Third' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'y' b'y' b'y' b'y']


# Working with continous data

In [8]:
# Pack into a vector

SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [31. 28. 28. 28. 28.]
n_siblings_spouses  : [0. 0. 0. 0. 0.]
parch               : [0. 0. 0. 0. 0.]
fare                : [8.683 7.854 7.896 7.75  7.55 ]


In [9]:
# Use iter to make it into a generator

example_batch, labels_batch = next(iter(temp_dataset)) 

In [10]:
# Pack into a vector

def pack(features, label):
  return tf.stack(list(features.values()), axis = -1), label

In [11]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[20.     0.     0.     9.5  ]
 [16.     0.     1.    39.4  ]
 [23.     0.     0.     7.896]
 [40.     1.     0.     9.475]
 [65.     0.     0.    26.55 ]]

[0 1 0 0 0]


In [12]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

# Numeric Data

In [13]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [14]:
show_batch(packed_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'female']
class               : [b'Third' b'Second' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'n' b'y' b'y' b'n']
numeric             : [[28.     1.     0.    16.1  ]
 [24.     1.     2.    65.   ]
 [16.     0.     0.     9.217]
 [28.     0.     0.     8.05 ]
 [50.     0.     1.    26.   ]]


In [15]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [16]:
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [17]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])
MEAN, STD

(array([29.631,  0.545,  0.38 , 34.385]),
 array([12.512,  1.151,  0.793, 54.598]))

In [18]:
# Same as sklearn.scaler.StandardScaler()

def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [19]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f5193c5edd0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [20]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[51.   ,  1.   ,  0.   , 77.958],
       [28.   ,  0.   ,  0.   , 12.35 ],
       [32.   ,  1.   ,  0.   , 15.85 ],
       [22.   ,  0.   ,  0.   ,  7.775],
       [28.   ,  1.   ,  0.   , 14.454]], dtype=float32)>

In [21]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 1.708,  0.395, -0.479,  0.798],
       [-0.13 , -0.474, -0.479, -0.404],
       [ 0.189,  0.395, -0.479, -0.339],
       [-0.61 , -0.474, -0.479, -0.487],
       [-0.13 ,  0.395, -0.479, -0.365]], dtype=float32)

# Categorical Data